In [44]:
import os
from dotenv import load_dotenv
load_dotenv()

API=os.environ.get("GROQ_API_KEY")

WEATHER_API = os.environ.get("WEATHER_API")

In [45]:
from langchain_core.tools import tool
import requests
from langchain_groq import ChatGroq
import warnings
warnings.filterwarnings("ignore")

In [46]:
# Use the Build-in tools
from langchain_community.utilities import ArxivAPIWrapper,WikipediaAPIWrapper
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun

arix_wraper =ArxivAPIWrapper(top_k_results=1)
arixve =ArxivQueryRun(api_wrapper=arix_wraper)

wiki_wraper =WikipediaAPIWrapper(top_k_results=2)
wiki = WikipediaQueryRun(api_wrapper=wiki_wraper)

#Create the custom tool
@tool
def weather_data(city:str)->str:
    """This function fetches the current weather for given city."""
    url = f'https://api.weatherstack.com/current?access_key={WEATHER_API}&query={city}'

    response = requests.get(url)
    return response.json()


In [47]:
from langchain.agents import create_react_agent,AgentExecutor
from langchain import hub
prompt = hub.pull("hwchase17/react")

In [48]:
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [49]:
llm = ChatGroq(model="llama-3.3-70b-versatile")
agent =create_react_agent(
    llm=llm,
    tools=[arixve,wiki,weather_data],
    prompt=prompt
)

In [51]:
agent_execute=AgentExecutor(
    agent=agent,
    tools=[arixve,wiki,weather_data],
    verbose=True
)

In [52]:
agent_execute.invoke({"input":"what is weather condition in karachi"})



> Entering new AgentExecutor chain...
Thought: To find the current weather condition in Karachi, I need to access a reliable source of weather information. The function weather_data(city: str) -> str seems to be the most suitable option for this task.

Action: weather_data
Action Input: Karachi{'request': {'type': 'City', 'query': 'Karachi, Pakistan', 'language': 'en', 'unit': 'm'}, 'location': {'name': 'Karachi', 'country': 'Pakistan', 'region': 'Sindh', 'lat': '24.867', 'lon': '67.050', 'timezone_id': 'Asia/Karachi', 'localtime': '2025-05-18 22:10', 'localtime_epoch': 1747606200, 'utc_offset': '5.0'}, 'current': {'observation_time': '05:10 PM', 'temperature': 30, 'weather_code': 143, 'weather_icons': ['https://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0006_mist.png'], 'weather_descriptions': ['Haze'], 'astro': {'sunrise': '05:47 AM', 'sunset': '07:11 PM', 'moonrise': 'No moonrise', 'moonset': '10:10 AM', 'moon_phase': 'Waning Gibbous', 'moon_illumination': 75}, 'a

{'input': 'what is weather condition in karachi',
 'output': 'The current weather condition in Karachi is Haze with a temperature of 30 degrees Celsius and a humidity of 66%.'}

In [35]:
res=agent_execute.invoke({"input":"Please give me top article on Machine learning 2025"})



> Entering new AgentExecutor chain...
To find the top article on Machine Learning for 2025, I should search for recent publications on Arxiv that are related to Machine Learning and have been published in 2025.

Action: arxiv
Action Input: Machine Learning 2025Published: 2025-02-11
Title: Singular leaning coefficients and efficiency in learning theory
Authors: Miki Aoyagi
Summary: Singular learning models with non-positive Fisher information matrices
include neural networks, reduced-rank regression, Boltzmann machines, normal
mixture models, and others. These models have been widely used in the
development of learning machines. However, theoretical analysis is still in its
early stages. In this paper, we examine learning coefficients, which indicate
the general learning efficiency of deep linear learning models and three-layer
neural network models with ReLU units. Finally, we extend the results to
include the case of the Softmax function.Thought: The article I found seems to be rela

In [36]:
print(res["output"])

The top article on Machine Learning for 2025 is "Progress in ${\cal CP}$ violating top-Higgs coupling at the LHC with Machine Learning" by A. Hammad and Adil Jueid, published on April 16, 2025. This article reviews recent progress in studies of the top-Higgs coupling at the Large Hadron Collider (LHC) and highlights the role of Machine Learning algorithms in constraining the ${\cal CP}$ phase of the top-Higgs coupling.
